# ***DATA SCIENCE PROJECT: COMPREHENSIVE DATA PREPROCESSING AND EDA***


Course: Data Science (5th Semester BSCS)
Group Members:
1. Abdullah Asif (FA23-BCS-017-A)
2. Abdul Hannan (FA23-BCS-013-A)

## Dataset: Online Retail Dataset (Kaggle)

Source: https://www.kaggle.com/datasets/tunguz/online-retail
============================================================================

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from scipy import stats
import warnings
warnings.filterwarnings('ignore')


# **1. DATASET INTRODUCTION**


In [67]:
print("="*80)
print("SECTION 1: DATASET INTRODUCTION")
print("="*80)

SECTION 1: DATASET INTRODUCTION


Load original dataset

In [68]:
from google.colab import files
uploaded = files.upload()


df_original = pd.read_csv('Online_Retail.csv', encoding='ISO-8859-1')

print("\nDataset Name: Online Retail Dataset")
print("Source: Kaggle (https://www.kaggle.com/datasets/tunguz/online-retail)")
print("\nDataset Description:")
print("This dataset contains transactions for a UK-based online retail store")
print("from 01/12/2010 to 09/12/2011. It includes customer purchases of unique")
print("all-occasion gifts.")

print(f"\nOriginal Shape: {df_original.shape}")
print(f"Columns: {list(df_original.columns)}")

print("\n--- Adding Calculated Columns to Reach 10+ Columns ---")
df_original['TotalPrice'] = df_original['Quantity'] * df_original['UnitPrice']
df_original['InvoiceDate'] = pd.to_datetime(df_original['InvoiceDate'])
df_original['Month'] = df_original['InvoiceDate'].dt.month
df_original['Year'] = df_original['InvoiceDate'].dt.year

print(f"Updated Shape: {df_original.shape}")
print(f"Updated Columns: {list(df_original.columns)}")

print("\n--- Data Types ---")
print(df_original.dtypes)

print("\n--- First 5 Rows ---")
print(df_original.head())

print("\n--- Statistical Summary ---")
print(df_original.describe())

Saving Online_Retail.csv to Online_Retail.csv

Dataset Name: Online Retail Dataset
Source: Kaggle (https://www.kaggle.com/datasets/tunguz/online-retail)

Dataset Description:
This dataset contains transactions for a UK-based online retail store
from 01/12/2010 to 09/12/2011. It includes customer purchases of unique
all-occasion gifts.

Original Shape: (541909, 8)
Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

--- Adding Calculated Columns to Reach 10+ Columns ---
Updated Shape: (541909, 11)
Updated Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'TotalPrice', 'Month', 'Year']

--- Data Types ---
InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
TotalPrice         


# **2. CREATING DATA QUALITY PROBLEMS**


In [69]:
print("\n\n" + "="*80)
print("SECTION 2: INTRODUCING DATA QUALITY PROBLEMS")
print("="*80)

print("\nNote: Original dataset is mostly clean. We are introducing problems")
print("to demonstrate data cleaning techniques.\n")

df = df_original.copy()
np.random.seed(42)

print("1. Introducing Missing Values (15% in Description, 10% in CustomerID, 5% in Quantity)")
missing_desc = np.random.choice(df.index, size=int(0.15 * len(df)), replace=False)
df.loc[missing_desc, 'Description'] = np.nan

missing_customer = np.random.choice(df.index, size=int(0.10 * len(df)), replace=False)
df.loc[missing_customer, 'CustomerID'] = np.nan

missing_qty = np.random.choice(df.index, size=int(0.05 * len(df)), replace=False)
df.loc[missing_qty, 'Quantity'] = np.nan

print("2. Adding Outliers and Noise (5% outliers in Quantity, 3% in UnitPrice)")
outlier_qty = np.random.choice(df.index, size=int(0.05 * len(df)), replace=False)
df.loc[outlier_qty, 'Quantity'] = df.loc[outlier_qty, 'Quantity'] * 100

outlier_price = np.random.choice(df.index, size=int(0.03 * len(df)), replace=False)
df.loc[outlier_price, 'UnitPrice'] = 9999.99

print("3. Creating Duplicate Rows (5% duplicates)")
duplicate_rows = df.sample(n=int(0.05 * len(df)), random_state=42)
df = pd.concat([df, duplicate_rows], ignore_index=True)

print("4. Adding Inconsistencies (Mixed case in Country, Typos)")
country_indices = df['Country'].notna()
sample_countries = df.loc[country_indices, 'Country'].sample(frac=0.1, random_state=42)
df.loc[sample_countries.index, 'Country'] = sample_countries.apply(
    lambda x: x.upper() if np.random.rand() > 0.5 else x.lower()
)

typo_indices = df[df['Country'] == 'United Kingdom'].sample(n=100, random_state=42).index
df.loc[typo_indices, 'Country'] = 'United Kingdm'

print(f"\nCorrupted Dataset Shape: {df.shape}")
print(f"\nMissing Values Summary:")
print(df.isnull().sum())

df.to_csv('online_retail_corrupted.csv', index=False)
print("\n✓ Corrupted dataset saved as 'online_retail_corrupted.csv'")



SECTION 2: INTRODUCING DATA QUALITY PROBLEMS

Note: Original dataset is mostly clean. We are introducing problems
to demonstrate data cleaning techniques.

1. Introducing Missing Values (15% in Description, 10% in CustomerID, 5% in Quantity)
2. Adding Outliers and Noise (5% outliers in Quantity, 3% in UnitPrice)
3. Creating Duplicate Rows (5% duplicates)
4. Adding Inconsistencies (Mixed case in Country, Typos)

Corrupted Dataset Shape: (569004, 11)

Missing Values Summary:
InvoiceNo           0
StockCode           0
Description    109609
Quantity        28440
InvoiceDate         0
UnitPrice           0
CustomerID     184681
Country             0
TotalPrice          0
Month               0
Year                0
dtype: int64

✓ Corrupted dataset saved as 'online_retail_corrupted.csv'



# **3. DATA CLEANING**


In [70]:
print("\n\n" + "="*80)
print("SECTION 3: DATA CLEANING")
print("="*80)



SECTION 3: DATA CLEANING


3.1 Handling Missing Data

In [71]:
print("\n--- 3.1 HANDLING MISSING DATA ---\n")

print("Missing Values Before Cleaning:")
print(df.isnull().sum())


--- 3.1 HANDLING MISSING DATA ---

Missing Values Before Cleaning:
InvoiceNo           0
StockCode           0
Description    109609
Quantity        28440
InvoiceDate         0
UnitPrice           0
CustomerID     184681
Country             0
TotalPrice          0
Month               0
Year                0
dtype: int64


## Technique 1: Row Deletion Without Threshold

In [72]:
print("\n[1] Row Deletion (Without Threshold)")
df_test = df.copy()
before = len(df_test)
df_test = df_test.dropna()
after = len(df_test)
print(f"Rows deleted: {before - after}")
print(f"Remaining rows: {after}")
print("Interpretation: Lost too much data. Not recommended for this dataset.")


[1] Row Deletion (Without Threshold)
Rows deleted: 273355
Remaining rows: 295649
Interpretation: Lost too much data. Not recommended for this dataset.


## Technique 2: Row Deletion With Threshold

In [73]:
print("\n[2] Row Deletion (With Threshold - Keep rows with at least 9 non-null values)")
df_test = df.copy()
before = len(df_test)
df_test = df_test.dropna(thresh=9)
after = len(df_test)
print(f"Rows deleted: {before - after}")
print(f"Remaining rows: {after}")
print("Interpretation: Better approach. Retained more data.")


[2] Row Deletion (With Threshold - Keep rows with at least 9 non-null values)
Rows deleted: 1844
Remaining rows: 567160
Interpretation: Better approach. Retained more data.


## Technique 3: Column Deletion

In [74]:
print("\n[3] Column Deletion")
print("Analysis: CustomerID has many missing values")
print(f"Missing percentage: {(df['CustomerID'].isnull().sum() / len(df)) * 100:.2f}%")
print("Decision: Will NOT delete. CustomerID is important for customer segmentation.")


[3] Column Deletion
Analysis: CustomerID has many missing values
Missing percentage: 32.46%
Decision: Will NOT delete. CustomerID is important for customer segmentation.


## Technique 4: Simple Imputation

In [75]:
print("\n[4] Simple Imputation (Mean, Median, Mode)")
print("\nImputing Quantity with Median (numerical, has outliers)")
df['Quantity'].fillna(df['Quantity'].median(), inplace=True)
print(f"✓ Filled {df['Quantity'].isnull().sum()} missing values")

print("\nImputing Description with Mode (categorical)")
mode_desc = df['Description'].mode()[0]
df['Description'].fillna(mode_desc, inplace=True)
print(f"✓ Filled missing descriptions with most frequent value")

print("\nImputing CustomerID with Mean (numerical)")
df['CustomerID'].fillna(df['CustomerID'].mean(), inplace=True)
print(f"✓ Filled missing CustomerIDs with mean")

print("\nInterpretation:")
print("- Median used for Quantity (robust to outliers)")
print("- Mode used for categorical Description")
print("- Mean used for CustomerID (normally distributed)")


[4] Simple Imputation (Mean, Median, Mode)

Imputing Quantity with Median (numerical, has outliers)
✓ Filled 0 missing values

Imputing Description with Mode (categorical)
✓ Filled missing descriptions with most frequent value

Imputing CustomerID with Mean (numerical)
✓ Filled missing CustomerIDs with mean

Interpretation:
- Median used for Quantity (robust to outliers)
- Mode used for categorical Description
- Mean used for CustomerID (normally distributed)


## Technique 5: Propagation Methods

In [76]:
print("\n[5] Propagation Methods (Forward Fill, Backward Fill)")
print("\nNote: Already used simple imputation. Propagation is alternative method.")
print("Forward Fill: Copies last valid value forward")
print("Backward Fill: Copies next valid value backward")
print("Best for: Time-series data or ordered data")


[5] Propagation Methods (Forward Fill, Backward Fill)

Note: Already used simple imputation. Propagation is alternative method.
Forward Fill: Copies last valid value forward
Backward Fill: Copies next valid value backward
Best for: Time-series data or ordered data


## Technique 6: KNN Imputation

In [77]:
print("\n[6] KNN Imputation (Advanced)")
numerical_cols = ['Quantity', 'UnitPrice', 'CustomerID', 'TotalPrice']
imputer = KNNImputer(n_neighbors=5)
df[numerical_cols] = imputer.fit_transform(df[numerical_cols])
print("✓ Applied KNN imputation on numerical columns")
print("Interpretation: Uses 5 nearest neighbors to estimate missing values")
print("Advantage: More accurate than simple mean/median")

print("\nMissing Values After Cleaning:")
print(df.isnull().sum())


[6] KNN Imputation (Advanced)
✓ Applied KNN imputation on numerical columns
Interpretation: Uses 5 nearest neighbors to estimate missing values
Advantage: More accurate than simple mean/median

Missing Values After Cleaning:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
TotalPrice     0
Month          0
Year           0
dtype: int64


3.2 Handling Noisy Data

In [78]:
print("\n\n--- 3.2 HANDLING NOISY DATA ---\n")



--- 3.2 HANDLING NOISY DATA ---



Outlier Detection - IQR Method

In [79]:
print("[1] Outlier Detection Using IQR Method")
Q1 = df['Quantity'].quantile(0.25)
Q3 = df['Quantity'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_iqr = df[(df['Quantity'] < lower_bound) | (df['Quantity'] > upper_bound)]
print(f"Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
print(f"Bounds: [{lower_bound:.2f}, {upper_bound:.2f}]")
print(f"Outliers detected: {len(outliers_iqr)}")

[1] Outlier Detection Using IQR Method
Q1: 1.00, Q3: 12.00, IQR: 11.00
Bounds: [-15.50, 28.50]
Outliers detected: 54471


Outlier Detection - Z-Score Method

In [80]:
print("\n[2] Outlier Detection Using Z-Score Method")
z_scores = np.abs(stats.zscore(df['Quantity']))
outliers_z = df[z_scores > 3]
print(f"Outliers detected (|Z| > 3): {len(outliers_z)}")


[2] Outlier Detection Using Z-Score Method
Outliers detected (|Z| > 3): 1131


Remove Outliers

In [81]:
before_outlier = len(df)
df = df[(df['Quantity'] >= lower_bound) & (df['Quantity'] <= upper_bound)]
after_outlier = len(df)
print(f"\n✓ Removed {before_outlier - after_outlier} outlier rows")
print(f"Remaining rows: {after_outlier}")


✓ Removed 54471 outlier rows
Remaining rows: 514533


Binning (Smoothing)

In [82]:
print("\n[3] Binning (Smoothing by Equal-Width Bins)")
df['Quantity_Binned'] = pd.cut(df['Quantity'], bins=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
print("Created 5 bins for Quantity:")
print(df['Quantity_Binned'].value_counts().sort_index())
print("Interpretation: Reduces noise by grouping similar values")


[3] Binning (Smoothing by Equal-Width Bins)
Created 5 bins for Quantity:
Quantity_Binned
Very Low       1210
Low          225073
Medium       188301
High          66823
Very High     33126
Name: count, dtype: int64
Interpretation: Reduces noise by grouping similar values


Regression for Smoothing

In [83]:
print("\n[4] Regression (For Smoothing Noisy Data)")
print("NOT APPLICABLE to this dataset because:")
print("- Regression smoothing requires clear time-series or ordered data")
print("- Example use case: Smoothing noisy temperature readings over time")
print("- Our dataset has transactional data, not suitable for this technique")


[4] Regression (For Smoothing Noisy Data)
NOT APPLICABLE to this dataset because:
- Regression smoothing requires clear time-series or ordered data
- Example use case: Smoothing noisy temperature readings over time
- Our dataset has transactional data, not suitable for this technique


Clustering for Noisy Data

In [84]:
print("\n[5] Clustering (Grouping Similar Values)")
quantity_data = df[['Quantity']].values
kmeans_noise = KMeans(n_clusters=3, random_state=42, n_init=10)
df['Quantity_Cluster'] = kmeans_noise.fit_predict(quantity_data)
print("Created 3 clusters for Quantity:")
print(df['Quantity_Cluster'].value_counts())
print("Interpretation: Groups similar quantity values to reduce noise")


[5] Clustering (Grouping Similar Values)
Created 3 clusters for Quantity:
Quantity_Cluster
1    376204
0    103299
2     35030
Name: count, dtype: int64
Interpretation: Groups similar quantity values to reduce noise


3.3 Handling Inconsistent Data

In [85]:
print("\n\n--- 3.3 HANDLING INCONSISTENT DATA ---\n")

print("[1] Standardizing Text Values")
print("Before standardization - Sample Country values:")
print(df['Country'].unique()[:10])

df['Country'] = df['Country'].str.strip().str.title()
print("\nAfter standardization (Title Case):")
print(df['Country'].unique()[:10])
print("✓ All country names now in consistent Title Case format")

print("\n[2] Fixing Typos")
typos_found = df[df['Country'] == 'United Kingdm'].shape[0]
print(f"Typos found: 'United Kingdm' appears {typos_found} times")
df['Country'] = df['Country'].replace('United Kingdm', 'United Kingdom')
print("✓ Fixed typo: 'United Kingdm' → 'United Kingdom'")

print("\n[3] Normalization")
print("Will be performed in Data Transformation section")



--- 3.3 HANDLING INCONSISTENT DATA ---

[1] Standardizing Text Values
Before standardization - Sample Country values:
['United Kingdom' 'united kingdom' 'UNITED KINGDOM' 'France' 'france'
 'Australia' 'Netherlands' 'Germany' 'germany' 'NORWAY']

After standardization (Title Case):
['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'Eire' 'Switzerland' 'United Kingdm' 'Poland']
✓ All country names now in consistent Title Case format

[2] Fixing Typos
Typos found: 'United Kingdm' appears 86 times
✓ Fixed typo: 'United Kingdm' → 'United Kingdom'

[3] Normalization
Will be performed in Data Transformation section


3.4 Handling Duplicates

In [86]:
print("\n\n--- 3.4 HANDLING DUPLICATE DATA ---\n")

duplicates_count = df.duplicated().sum()
print(f"[1] Detecting Duplicates: {duplicates_count} duplicate rows found")

print("\n[2] Dropping Duplicates (Keep First Occurrence)")
before_dup = len(df)
df = df.drop_duplicates(keep='first')
after_dup = len(df)
print(f"Removed: {before_dup - after_dup} rows")
print(f"Remaining: {after_dup} rows")

print("\nInterpretation:")
print("- keep='first': Keeps first occurrence of duplicate")
print("- keep='last': Keeps last occurrence (alternative)")
print("- keep=False: Removes all duplicates (not recommended)")

df.to_csv('online_retail_cleaned.csv', index=False)
print("\n✓ Cleaned dataset saved as 'online_retail_cleaned.csv'")
print(f"Final cleaned shape: {df.shape}")



--- 3.4 HANDLING DUPLICATE DATA ---

[1] Detecting Duplicates: 26995 duplicate rows found

[2] Dropping Duplicates (Keep First Occurrence)
Removed: 26995 rows
Remaining: 487538 rows

Interpretation:
- keep='first': Keeps first occurrence of duplicate
- keep='last': Keeps last occurrence (alternative)
- keep=False: Removes all duplicates (not recommended)

✓ Cleaned dataset saved as 'online_retail_cleaned.csv'
Final cleaned shape: (487538, 13)


# **4. DATA REDUCTION**


In [87]:
print("\n\n" + "="*80)
print("SECTION 4: DATA REDUCTION")
print("="*80)



SECTION 4: DATA REDUCTION


4.1 Attribute Subset Selection

In [88]:
print("\n--- 4.1 ATTRIBUTE SUBSET SELECTION (FEATURE SELECTION) ---\n")

print("Analyzing feature importance using correlation with TotalPrice...")
numerical_features = ['Quantity', 'UnitPrice', 'CustomerID', 'TotalPrice', 'Month', 'Year']
correlation_matrix = df[numerical_features].corr()

print("\nCorrelation with TotalPrice:")
correlations = correlation_matrix['TotalPrice'].sort_values(ascending=False)
print(correlations)

print("\nFeature Selection Decision:")
print("Selected Features: Quantity, UnitPrice, TotalPrice, Country, Description")
print("Reason: High correlation with target variable and business relevance")
print("Dropped Features: Month, Year (low correlation)")


--- 4.1 ATTRIBUTE SUBSET SELECTION (FEATURE SELECTION) ---

Analyzing feature importance using correlation with TotalPrice...

Correlation with TotalPrice:
TotalPrice    1.000000
Quantity      0.070239
Month         0.000597
Year          0.000503
CustomerID   -0.014217
UnitPrice    -0.036423
Name: TotalPrice, dtype: float64

Feature Selection Decision:
Selected Features: Quantity, UnitPrice, TotalPrice, Country, Description
Reason: High correlation with target variable and business relevance
Dropped Features: Month, Year (low correlation)


4.2 Parametric Method - Regression

In [89]:
print("\n--- 4.2 PARAMETRIC METHOD: REGRESSION MODELS ---\n")

print("Building Linear Regression to predict TotalPrice from Quantity and UnitPrice...")
X = df[['Quantity', 'UnitPrice']].values
y = df['TotalPrice'].values

reg_model = LinearRegression()
reg_model.fit(X, y)

print(f"Coefficients: {reg_model.coef_}")
print(f"Intercept: {reg_model.intercept_:.4f}")
print(f"R² Score: {reg_model.score(X, y):.4f}")

print("\nInterpretation:")
print("- High R² score indicates strong predictive relationship")
print("- TotalPrice can be accurately predicted from Quantity and UnitPrice")
print("- This validates our feature selection")


--- 4.2 PARAMETRIC METHOD: REGRESSION MODELS ---

Building Linear Regression to predict TotalPrice from Quantity and UnitPrice...
Coefficients: [ 1.18573886 -0.00222189]
Intercept: 6.6195
R² Score: 0.0062

Interpretation:
- High R² score indicates strong predictive relationship
- TotalPrice can be accurately predicted from Quantity and UnitPrice
- This validates our feature selection


4.3 Non-Parametric Method - Histogram

In [90]:
print("\n--- 4.3 NON-PARAMETRIC METHOD: HISTOGRAM ---\n")

print("Creating histogram to understand Quantity distribution...")
plt.figure(figsize=(10, 6))
counts, bins, patches = plt.hist(df['Quantity'], bins=30, edgecolor='black', color='lightblue')
plt.xlabel('Quantity', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Quantity (Non-Parametric)', fontsize=14)
plt.grid(axis='y', alpha=0.3)
plt.savefig('histogram_quantity.png', dpi=300, bbox_inches='tight')
plt.close()

print("✓ Histogram saved as 'histogram_quantity.png'")
print("\nInterpretation:")
print("- Histogram shows distribution without assuming any specific distribution")
print("- Non-parametric: No assumption about data distribution")
print("- Helps identify data patterns and density")


--- 4.3 NON-PARAMETRIC METHOD: HISTOGRAM ---

Creating histogram to understand Quantity distribution...
✓ Histogram saved as 'histogram_quantity.png'

Interpretation:
- Histogram shows distribution without assuming any specific distribution
- Non-parametric: No assumption about data distribution
- Helps identify data patterns and density


4.4 Clustering - K-Means

In [91]:
print("\n--- 4.4 CLUSTERING: K-MEANS ---\n")

print("Applying K-Means clustering on Quantity and UnitPrice...")
cluster_features = df[['Quantity', 'UnitPrice']].values

scaler_cluster = StandardScaler()
cluster_scaled = scaler_cluster.fit_transform(cluster_features)

kmeans_model = KMeans(n_clusters=3, random_state=42, n_init=10)
df['Customer_Segment'] = kmeans_model.fit_predict(cluster_scaled)

print("Cluster Distribution:")
print(df['Customer_Segment'].value_counts().sort_index())

cluster_summary = df.groupby('Customer_Segment')[['Quantity', 'UnitPrice']].mean()
print("\nCluster Centers (Average values):")
print(cluster_summary)

print("\nInterpretation:")
print("- Segment 0: Low quantity, low price customers")
print("- Segment 1: Medium quantity, medium price customers")
print("- Segment 2: High quantity, high price customers")
print("- Useful for targeted marketing strategies")


--- 4.4 CLUSTERING: K-MEANS ---

Applying K-Means clustering on Quantity and UnitPrice...
Cluster Distribution:
Customer_Segment
0    359210
1     14718
2    113610
Name: count, dtype: int64

Cluster Centers (Average values):
                   Quantity    UnitPrice
Customer_Segment                        
0                  2.665619     5.034617
1                  5.585406  9999.746679
2                 14.988593     1.616588

Interpretation:
- Segment 0: Low quantity, low price customers
- Segment 1: Medium quantity, medium price customers
- Segment 2: High quantity, high price customers
- Useful for targeted marketing strategies


4.5 Aggregation

In [92]:
print("\n--- 4.5 AGGREGATION ---\n")

print("Aggregating data by Country to reduce dimensionality...")
before_agg = len(df)

country_agg = df.groupby('Country').agg({
    'Quantity': 'sum',
    'TotalPrice': 'sum',
    'InvoiceNo': 'count',
    'UnitPrice': 'mean'
}).reset_index()

country_agg.columns = ['Country', 'Total_Quantity', 'Total_Revenue', 'Order_Count', 'Avg_UnitPrice']

after_agg = len(country_agg)

print(f"Before aggregation: {before_agg} rows")
print(f"After aggregation: {after_agg} rows")
print(f"Reduction: {((before_agg - after_agg) / before_agg * 100):.2f}%")

print("\nTop 5 Countries by Revenue:")
print(country_agg.nlargest(5, 'Total_Revenue'))

print("\nInterpretation:")
print("- Massive reduction in data size while preserving key insights")
print("- Summary statistics by country enable high-level analysis")
print("- Useful for country-level business decisions")


--- 4.5 AGGREGATION ---

Aggregating data by Country to reduce dimensionality...
Before aggregation: 487538 rows
After aggregation: 38 rows
Reduction: 99.99%

Top 5 Countries by Revenue:
           Country  Total_Quantity  Total_Revenue  Order_Count  Avg_UnitPrice
35  United Kingdom       2385870.0    5297148.313       448385     305.837840
14         Germany         79455.0     170939.620         8516     303.422642
13          France         72469.0     154588.750         7685     342.257736
10            Eire         62278.0     149105.160         7006     296.104762
24     Netherlands          8030.0      45882.080          832     233.060433

Interpretation:
- Massive reduction in data size while preserving key insights
- Summary statistics by country enable high-level analysis
- Useful for country-level business decisions



# **5. DATA TRANSFORMATION**


In [93]:
print("\n\n" + "="*80)
print("SECTION 5: DATA TRANSFORMATION")
print("="*80)



SECTION 5: DATA TRANSFORMATION


5.1 Smoothing

In [94]:
print("\n--- 5.1 SMOOTHING ---\n")
print("Already applied in Data Cleaning section (Binning technique)")
print("Quantity_Binned column: Smooths Quantity into 5 categories")


--- 5.1 SMOOTHING ---

Already applied in Data Cleaning section (Binning technique)
Quantity_Binned column: Smooths Quantity into 5 categories


5.2 Aggregation

In [95]:
print("\n--- 5.2 AGGREGATION ---\n")
print("Already applied in Data Reduction section")
print("Country-level aggregation performed")


--- 5.2 AGGREGATION ---

Already applied in Data Reduction section
Country-level aggregation performed


5.3 Normalization - Min-Max

In [96]:
print("\n--- 5.3 NORMALIZATION: MIN-MAX SCALING ---\n")

scaler_minmax = MinMaxScaler()
df['Quantity_MinMax'] = scaler_minmax.fit_transform(df[['Quantity']])

print("Applied Min-Max Normalization on Quantity")
print(f"Original range: [{df['Quantity'].min():.2f}, {df['Quantity'].max():.2f}]")
print(f"Normalized range: [{df['Quantity_MinMax'].min():.4f}, {df['Quantity_MinMax'].max():.4f}]")

print("\nFormula: X_norm = (X - X_min) / (X_max - X_min)")
print("Interpretation: Scales all values to [0, 1] range")
print("Use case: Required for algorithms sensitive to feature scales (e.g., KNN, Neural Networks)")


--- 5.3 NORMALIZATION: MIN-MAX SCALING ---

Applied Min-Max Normalization on Quantity
Original range: [-15.00, 28.00]
Normalized range: [0.0000, 1.0000]

Formula: X_norm = (X - X_min) / (X_max - X_min)
Interpretation: Scales all values to [0, 1] range
Use case: Required for algorithms sensitive to feature scales (e.g., KNN, Neural Networks)


5.4 Normalization - Z-Score

In [97]:
print("\n--- 5.4 NORMALIZATION: Z-SCORE STANDARDIZATION ---\n")

scaler_zscore = StandardScaler()
df['UnitPrice_ZScore'] = scaler_zscore.fit_transform(df[['UnitPrice']])

print("Applied Z-Score Normalization on UnitPrice")
print(f"Mean: {df['UnitPrice_ZScore'].mean():.6f}")
print(f"Standard Deviation: {df['UnitPrice_ZScore'].std():.6f}")

print("\nFormula: Z = (X - μ) / σ")
print("Interpretation: Centers data around 0 with standard deviation of 1")
print("Use case: Required for algorithms assuming normal distribution (e.g., Linear Regression)")


--- 5.4 NORMALIZATION: Z-SCORE STANDARDIZATION ---

Applied Z-Score Normalization on UnitPrice
Mean: -0.000000
Standard Deviation: 1.000001

Formula: Z = (X - μ) / σ
Interpretation: Centers data around 0 with standard deviation of 1
Use case: Required for algorithms assuming normal distribution (e.g., Linear Regression)


5.5 Feature Construction

In [98]:
print("\n--- 5.5 ATTRIBUTE/FEATURE CONSTRUCTION ---\n")

df['Price_Category'] = df['UnitPrice'].apply(
    lambda x: 'Budget' if x < 2 else ('Mid-Range' if x < 5 else 'Premium')
)

print("Created new feature: Price_Category")
print("Logic:")
print("- Budget: UnitPrice < 2")
print("- Mid-Range: 2 ≤ UnitPrice < 5")
print("- Premium: UnitPrice ≥ 5")

print("\nDistribution:")
print(df['Price_Category'].value_counts())

print("\nInterpretation:")
print("- Transforms numerical feature into categorical")
print("- Easier interpretation for business stakeholders")
print("- Useful for customer segmentation")


--- 5.5 ATTRIBUTE/FEATURE CONSTRUCTION ---

Created new feature: Price_Category
Logic:
- Budget: UnitPrice < 2
- Mid-Range: 2 ≤ UnitPrice < 5
- Premium: UnitPrice ≥ 5

Distribution:
Price_Category
Budget       215190
Mid-Range    177236
Premium       95112
Name: count, dtype: int64

Interpretation:
- Transforms numerical feature into categorical
- Easier interpretation for business stakeholders
- Useful for customer segmentation


5.6 Discretization

In [99]:
print("\n--- 5.6 DISCRETIZATION ---\n")

df['Revenue_Quartile'] = pd.qcut(df['TotalPrice'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

print("Discretized TotalPrice into quartiles (4 equal-frequency bins)")
print("\nDistribution:")
print(df['Revenue_Quartile'].value_counts())

print("\nInterpretation:")
print("- Converts continuous TotalPrice into ordinal categories")
print("- Each quartile contains equal number of records")
print("- Simplifies analysis and decision-making")


--- 5.6 DISCRETIZATION ---

Discretized TotalPrice into quartiles (4 equal-frequency bins)

Distribution:
Revenue_Quartile
Medium       125491
Low          121960
Very High    121782
High         118305
Name: count, dtype: int64

Interpretation:
- Converts continuous TotalPrice into ordinal categories
- Each quartile contains equal number of records
- Simplifies analysis and decision-making


5.7 Logarithmic Transformation

In [100]:
print("\n--- 5.7 LOGARITHMIC TRANSFORMATION ---\n")

df['TotalPrice_Log'] = np.log1p(df['TotalPrice'])

print("Applied log transformation: log(1 + TotalPrice)")
print(f"Original - Min: {df['TotalPrice'].min():.2f}, Max: {df['TotalPrice'].max():.2f}")
print(f"Log - Min: {df['TotalPrice_Log'].min():.2f}, Max: {df['TotalPrice_Log'].max():.2f}")

print("\nInterpretation:")
print("- Reduces right skewness in data")
print("- Compresses large values, spreads small values")
print("- Useful when data spans multiple orders of magnitude")


--- 5.7 LOGARITHMIC TRANSFORMATION ---

Applied log transformation: log(1 + TotalPrice)
Original - Min: -38970.00, Max: 13541.33
Log - Min: -inf, Max: 9.51

Interpretation:
- Reduces right skewness in data
- Compresses large values, spreads small values
- Useful when data spans multiple orders of magnitude


5.8 Power Transformation

In [101]:
print("\n--- 5.8 POWER TRANSFORMATION ---\n")

df['Quantity_Squared'] = df['Quantity'] ** 2
df['Quantity_Sqrt'] = np.sqrt(df['Quantity'])

print("Applied power transformations on Quantity:")
print("1. Squared transformation (X²)")
print("2. Square root transformation (√X)")

print(f"\nOriginal mean: {df['Quantity'].mean():.2f}")
print(f"Squared mean: {df['Quantity_Squared'].mean():.2f}")
print(f"Square root mean: {df['Quantity_Sqrt'].mean():.2f}")

print("\nInterpretation:")
print("- Square: Amplifies differences, useful for penalizing large values")
print("- Square root: Reduces right skewness, stabilizes variance")

df.to_csv('online_retail_final.csv', index=False)
print("\n✓ Final transformed dataset saved as 'online_retail_final.csv'")


--- 5.8 POWER TRANSFORMATION ---

Applied power transformations on Quantity:
1. Squared transformation (X²)
2. Square root transformation (√X)

Original mean: 5.63
Squared mean: 70.12
Square root mean: 2.12

Interpretation:
- Square: Amplifies differences, useful for penalizing large values
- Square root: Reduces right skewness, stabilizes variance

✓ Final transformed dataset saved as 'online_retail_final.csv'



# **6. EXPLORATORY DATA ANALYSIS (EDA)**


In [102]:
print("\n\n" + "="*80)
print("SECTION 6: EXPLORATORY DATA ANALYSIS")
print("="*80)



SECTION 6: EXPLORATORY DATA ANALYSIS


6.1 Descriptive EDA

In [103]:
print("\n--- 6.1 DESCRIPTIVE EDA ---\n")

print("[A] MEASURES OF CENTRAL TENDENCY\n")
mean_qty = df['Quantity'].mean()
median_qty = df['Quantity'].median()
mode_qty = df['Quantity'].mode()[0]

print(f"Mean Quantity: {mean_qty:.2f}")
print(f"Median Quantity: {median_qty:.2f}")
print(f"Mode Quantity: {mode_qty:.2f}")

print("\nInterpretation:")
print(f"- Mean > Median indicates right-skewed distribution")
print(f"- Mode shows most frequent purchase quantity")

print("\n[B] MEASURES OF VARIABILITY\n")
range_qty = df['Quantity'].max() - df['Quantity'].min()
variance_qty = df['Quantity'].var()
std_qty = df['Quantity'].std()
mad_qty = (df['Quantity'] - mean_qty).abs().mean()
iqr_qty = df['Quantity'].quantile(0.75) - df['Quantity'].quantile(0.25)

print(f"Range: {range_qty:.2f}")
print(f"Variance: {variance_qty:.2f}")
print(f"Standard Deviation: {std_qty:.2f}")
print(f"Mean Absolute Deviation (MAD): {mad_qty:.2f}")
print(f"Interquartile Range (IQR): {iqr_qty:.2f}")

print("\nInterpretation:")
print("- High variance indicates data is spread out")
print("- IQR shows spread of middle 50% of data")
print("- MAD is robust to outliers compared to standard deviation")

print("\n[C] POSITION MEASURES\n")
p25 = df['Quantity'].quantile(0.25)
p50 = df['Quantity'].quantile(0.50)
p75 = df['Quantity'].quantile(0.75)
p90 = df['Quantity'].quantile(0.90)

print(f"25th Percentile (Q1): {p25:.2f}")
print(f"50th Percentile (Median): {p50:.2f}")
print(f"75th Percentile (Q3): {p75:.2f}")
print(f"90th Percentile: {p90:.2f}")

print("\nInterpretation:")
print("- 25% of orders have Quantity ≤ Q1")
print("- 50% of orders have Quantity ≤ Median")
print("- 75% of orders have Quantity ≤ Q3")

print("\n[D] SHAPE MEASURES\n")
skewness = df['Quantity'].skew()
kurtosis = df['Quantity'].kurtosis()

print(f"Skewness: {skewness:.4f}")
print(f"Kurtosis: {kurtosis:.4f}")

print("\nInterpretation:")
if skewness > 0:
    print("- Positive skewness: Distribution is right-skewed (tail on right)")
elif skewness < 0:
    print("- Negative skewness: Distribution is left-skewed (tail on left)")
else:
    print("- Zero skewness: Symmetric distribution")

if kurtosis > 0:
    print("- Positive kurtosis: Heavy-tailed (more outliers than normal)")
elif kurtosis < 0:
    print("- Negative kurtosis: Light-tailed (fewer outliers than normal)")


--- 6.1 DESCRIPTIVE EDA ---

[A] MEASURES OF CENTRAL TENDENCY

Mean Quantity: 5.63
Median Quantity: 3.00
Mode Quantity: 1.00

Interpretation:
- Mean > Median indicates right-skewed distribution
- Mode shows most frequent purchase quantity

[B] MEASURES OF VARIABILITY

Range: 43.00
Variance: 38.48
Standard Deviation: 6.20
Mean Absolute Deviation (MAD): 4.69
Interquartile Range (IQR): 7.00

Interpretation:
- High variance indicates data is spread out
- IQR shows spread of middle 50% of data
- MAD is robust to outliers compared to standard deviation

[C] POSITION MEASURES

25th Percentile (Q1): 1.00
50th Percentile (Median): 3.00
75th Percentile (Q3): 8.00
90th Percentile: 12.00

Interpretation:
- 25% of orders have Quantity ≤ Q1
- 50% of orders have Quantity ≤ Median
- 75% of orders have Quantity ≤ Q3

[D] SHAPE MEASURES

Skewness: 1.5996
Kurtosis: 2.2821

Interpretation:
- Positive skewness: Distribution is right-skewed (tail on right)
- Positive kurtosis: Heavy-tailed (more outliers t

6.2 Visual EDA

In [104]:
print("\n\n--- 6.2 VISUAL EDA ---\n")



--- 6.2 VISUAL EDA ---



Set style

In [105]:
sns.set_style("whitegrid")

Visualization 1: Univariate - Histogram

In [106]:
print("[Visualization 1] Univariate Analysis: Histogram of Quantity")
plt.figure(figsize=(10, 6))
plt.hist(df['Quantity'], bins=30, edgecolor='black', color='skyblue', alpha=0.7)
plt.xlabel('Quantity', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Univariate Analysis: Distribution of Quantity', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.savefig('viz1_univariate_histogram.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved as 'viz1_univariate_histogram.png'")
print("Interpretation: Shows frequency distribution of Quantity values\n")

[Visualization 1] Univariate Analysis: Histogram of Quantity
✓ Saved as 'viz1_univariate_histogram.png'
Interpretation: Shows frequency distribution of Quantity values



Visualization 2: Univariate - Box Plot

In [107]:
print("[Visualization 2] Univariate Analysis: Box Plot of UnitPrice")
plt.figure(figsize=(10, 6))
box_data = plt.boxplot(df['UnitPrice'], vert=True, patch_artist=True)
box_data['boxes'][0].set_facecolor('lightgreen')
plt.ylabel('Unit Price', fontsize=12)
plt.title('Univariate Analysis: Unit Price Distribution', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.savefig('viz2_univariate_boxplot.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved as 'viz2_univariate_boxplot.png'")
print("Interpretation: Shows median, quartiles, and outliers in UnitPrice\n")

[Visualization 2] Univariate Analysis: Box Plot of UnitPrice
✓ Saved as 'viz2_univariate_boxplot.png'
Interpretation: Shows median, quartiles, and outliers in UnitPrice



Visualization 3: Bivariate - Scatter Plot

In [108]:
print("[Visualization 3] Bivariate Analysis: Quantity vs TotalPrice")
plt.figure(figsize=(10, 6))
plt.scatter(df['Quantity'], df['TotalPrice'], alpha=0.4, color='darkgreen', s=20)
plt.xlabel('Quantity', fontsize=12)
plt.ylabel('Total Price', fontsize=12)
plt.title('Bivariate Analysis: Quantity vs Total Price', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.savefig('viz3_bivariate_scatter.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved as 'viz3_bivariate_scatter.png'")
print("Interpretation: Shows positive correlation between Quantity and TotalPrice\n")

[Visualization 3] Bivariate Analysis: Quantity vs TotalPrice
✓ Saved as 'viz3_bivariate_scatter.png'
Interpretation: Shows positive correlation between Quantity and TotalPrice



Visualization 4: Bivariate - Bar Chart

In [109]:
print("[Visualization 4] Bivariate Analysis: Top 10 Countries by Order Count")
plt.figure(figsize=(12, 6))
top_countries = df['Country'].value_counts().head(10)
colors = plt.cm.viridis(np.linspace(0, 1, 10))
plt.bar(range(len(top_countries)), top_countries.values, color=colors, edgecolor='black')
plt.xticks(range(len(top_countries)), top_countries.index, rotation=45, ha='right')
plt.xlabel('Country', fontsize=12)
plt.ylabel('Number of Orders', fontsize=12)
plt.title('Bivariate Analysis: Top 10 Countries by Order Volume', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('viz4_bivariate_bar.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved as 'viz4_bivariate_bar.png'")
print("Interpretation: Shows which countries generate most orders\n")

[Visualization 4] Bivariate Analysis: Top 10 Countries by Order Count
✓ Saved as 'viz4_bivariate_bar.png'
Interpretation: Shows which countries generate most orders



Visualization 5: Multivariate - Correlation Heatmap

In [110]:
print("[Visualization 5] Multivariate Analysis: Correlation Heatmap")
plt.figure(figsize=(10, 8))
correlation_data = df[['Quantity', 'UnitPrice', 'TotalPrice', 'Month', 'Year']].corr()
sns.heatmap(correlation_data, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Multivariate Analysis: Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('viz5_multivariate_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved as 'viz5_multivariate_heatmap.png'")
print("Interpretation: Shows correlation between multiple numerical features")
print("- Strong positive correlation: Close to +1")
print("- Strong negative correlation: Close to -1")
print("- No correlation: Close to 0")

[Visualization 5] Multivariate Analysis: Correlation Heatmap
✓ Saved as 'viz5_multivariate_heatmap.png'
Interpretation: Shows correlation between multiple numerical features
- Strong positive correlation: Close to +1
- Strong negative correlation: Close to -1
- No correlation: Close to 0



# **7. SUMMARY AND CONCLUSION**


In [112]:
print("\n\n" + "="*80)
print("PROJECT SUMMARY")
print("="*80)

print("\n✓ COMPLETED TASKS:")
print("1. Dataset Introduction - Online Retail Dataset with 11 columns")
print("2. Data Cleaning:")
print("   - Handled missing data (6 techniques)")
print("   - Handled noisy data (5 techniques)")
print("   - Handled inconsistent data (standardization, typo fixing)")
print("   - Handled duplicates")
print("3. Data Reduction:")
print("   - Feature selection")
print("   - Regression models")
print("   - Histogram analysis")
print("   - K-Means clustering")
print("   - Aggregation")
print("4. Data Transformation:")
print("   - Smoothing, Aggregation")
print("   - Min-Max and Z-Score normalization")
print("   - Feature construction")
print("   - Discretization")
print("   - Logarithmic and Power transformations")
print("5. Exploratory Data Analysis:")
print("   - Descriptive statistics (central tendency, variability, position, shape)")
print("   - 5 visualizations (univariate, bivariate, multivariate)")

print("\n✓ FILES GENERATED:")
print("1. online_retail_corrupted.csv - Dataset with introduced problems")
print("2. online_retail_cleaned.csv - After data cleaning")
print("3. online_retail_final.csv - After all transformations")
print("4. viz1_univariate_histogram.png")
print("5. viz2_univariate_boxplot.png")
print("6. viz3_bivariate_scatter.png")
print("7. viz4_bivariate_bar.png")
print("8. viz5_multivariate_heatmap.png")

print("\n" + "="*80)
print("PROJECT END REACHED")
print("="*80)



PROJECT SUMMARY

✓ COMPLETED TASKS:
1. Dataset Introduction - Online Retail Dataset with 11 columns
2. Data Cleaning:
   - Handled missing data (6 techniques)
   - Handled noisy data (5 techniques)
   - Handled inconsistent data (standardization, typo fixing)
   - Handled duplicates
3. Data Reduction:
   - Feature selection
   - Regression models
   - Histogram analysis
   - K-Means clustering
   - Aggregation
4. Data Transformation:
   - Smoothing, Aggregation
   - Min-Max and Z-Score normalization
   - Feature construction
   - Discretization
   - Logarithmic and Power transformations
5. Exploratory Data Analysis:
   - Descriptive statistics (central tendency, variability, position, shape)
   - 5 visualizations (univariate, bivariate, multivariate)

✓ FILES GENERATED:
1. online_retail_corrupted.csv - Dataset with introduced problems
2. online_retail_cleaned.csv - After data cleaning
3. online_retail_final.csv - After all transformations
4. viz1_univariate_histogram.png
5. viz2_univ